## **RAG의 핵심, 문서 검색기 Retriever**

### **Retriever의 기본형, 벡터DB 기반 Retriever**

**Chroma 벡터 DB 기반 기본 유사 문서 검색**

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"C:\Users\gram\Downloads\대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

#Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

[Document(page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.', metadata={'page': 4, 'source': 'C:\\Users\\gram\\Downloads\\대한민국헌법(헌법)(제00010호)(19880225).pdf'}),
 Document(page_content='법제처                                                            5                                                       국가법령정보센터\n대한민국헌법 \n③국회의원의 선거구와 비례대표제 기타 선거에 관한 사항은 법률로 정한다.\n \n제42조 국회의원의 임기는 4년으로 한다.\n \n제43조 국회의원은 법률이 정하는 직을 겸할 수 없다.\n \n제44조 ①국회의원은 현행범인인 경우를 제외하고는 회기 중 국회의 동의없이 체포 또는 구금되지 아니한다.\n②국회의원이 회기 전에 체포 또는 구금된 때에는 현행범인이 아닌 한 

**(1) 유사도 점수도 함께 출력하기**

In [4]:
result_score = db.similarity_search_with_score("국회의원의 의무")
result_r_score = db.similarity_search_with_relevance_scores("국회의원의 의무")

print("[유사 청크 1순위]")
print(result_score[0][0].page_content)

print("\n\n[점수]")
print(result_score[0][1])
print(result_r_score[0][1])

[유사 청크 1순위]
③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.


[점수]
0.8005110025405884
0.4339532416891084


**(2) 검색 결과 수 및 조정**

In [5]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 1})
retriever.get_relevant_documents("국회의원의 의무")

C:\Users\gram\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.', metadata={'page': 4, 'source': 'C:\\Users\\gram\\Downloads\\대한민국헌법(헌법)(제00010호)(19880225).pdf'})]

**(3) 검색 방식 변경 - MMR**

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#헌법 PDF 파일 로드
loader = PyPDFLoader(r"C:\Users\gram\Downloads\대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
retriever.invoke("국회의원의 의무")

**일반 유사도 검색 방식**

In [ ]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
retriever.get_relevant_documents("국회의원의 의무")


### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**

**Chroma DB에 문서 벡터 저장**

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
#헌법 PDF 파일 로드
loader = PyPDFLoader(r"C:\Users\gram\Downloads\대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [7]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 국회의원이 가져야 하는 책임은 무엇인가요?', '2. 국회의원은 어떤 의무를 부담하고 있나요?', '3. 국회의원이 수행해야 하는 역할에는 어떤 것들이 있나요?']


4

### **문서를 여러 벡터로 재해석하다, MultiVectorRetriever**

**Chroma DB에 문서 벡터 저장**

In [8]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = PyPDFLoader(r"C:\Users\gram\Downloads\대한민국헌법(헌법)(제00010호)(19880225).pdf"),
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

**Multi Vector를 만들기 위한 작업**

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# 상위 문서 저장 위한 레이어 선언
store = InMemoryByteStore()
id_key = "doc_id"
# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

#문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [10]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)
#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [11]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.get_relevant_documents("국민의 의무")[0].page_content)

[하위 청크] 

③공공필요에 의한 재산권의 수용ㆍ사용 또는 제한 및 그에 대한 보상은 법률로써 하되, 정당한 보상을 지급하여야
한다.
 
제24조 모든 국민은 법률이 정하는 바에 의하여 선거권을 가진다.
 
제25조 모든 국민은 법률이 정하는 바에 의하여 공무담임권을 가진다.
 
제26조 ①모든 국민은 법률이 정하는 바에 의하여 국가기관에 문서로 청원할 권리를 가진다.
②국가는 청원에 대하여 심사할 의무를 진다.
 
제27조 ①모든 국민은 헌법과 법률이 정한 법관에 의하여 법률에 의한 재판을 받을 권리를 가진다.
②군인 또는 군무원이 아닌 국민은 대한민국의 영역 안에서는 중대한 군사상 기밀ㆍ초병ㆍ초소ㆍ유독음식물공급
ㆍ포로ㆍ군용물에 관한 죄중 법률이 정한 경우와 비상계엄이 선포된 경우를 제외하고는 군사법원의 재판을 받지
--------------------------------------------------
[상위 청크] 

법제처                                                            4                                                       국가법령정보센터
대한민국헌법 
제32조 ①모든 국민은 근로의 권리를 가진다. 국가는 사회적ㆍ경제적 방법으로 근로자의 고용의 증진과 적정임금의 보
장에 노력하여야 하며, 법률이 정하는 바에 의하여 최저임금제를 시행하여야 한다.
②모든 국민은 근로의 의무를 진다. 국가는 근로의 의무의 내용과 조건을 민주주의원칙에 따라 법률로 정한다.
③근로조건의 기준은 인간의 존엄성을 보장하도록 법률로 정한다.
④여자의 근로는 특별한 보호를 받으며, 고용ㆍ임금 및 근로조건에 있어서 부당한 차별을 받지 아니한다.
⑤연소자의 근로는 특별한 보호를 받는다.
⑥국가유공자ㆍ상이군경 및 전몰군경의 유가족은 법률이 정하는 바에 의하여 우선적으로 근로의 기회를 부여받는
다.
 
제33조 ①근로자는 근로조건의 향상을 위하여 자주적인 단결권ㆍ단체교섭권 및 단체행동권

### **컨텍스트 재정렬, Long-Context Reorder**

**[Long-Context Reorder 없이 유사 문서 출력]**

In [13]:
#Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

In [15]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(page_content='바스켓볼은 훌륭한 스포츠입니다.')]

**[Long-Context Reorder 활용하여 유사 문서 출력]**

In [16]:
#LongContextReorder 선언
reordering = LongContextReorder()
#검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs 

[Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.')]

In [14]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]
# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding=embedding).as_retriever(
    search_kwargs={"k": 10}
)
query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(page_content='래리 버드는 상징적인 NBA 선수였습니다.')]